In [1]:
using CSV, DataFrames, JSON, DataDrop

In [2]:
using SparseArrays
using LinearAlgebra

## Network

In [3]:
network = JSON.parsefile("../models/europe.json")

Dict{String, Any} with 11 entries:
  "bus"      => Dict{String, Any}("4304"=>Dict{String, Any}("coord"=>Any[11.843…
  "name"     => "pantagruel"
  "dcline"   => Dict{String, Any}()
  "gen"      => Dict{String, Any}("1"=>Dict{String, Any}("entsoe_names"=>Any[],…
  "branch"   => Dict{String, Any}("4304"=>Dict{String, Any}("br_r"=>0.00174284,…
  "storage"  => Dict{String, Any}()
  "switch"   => Dict{String, Any}()
  "baseMVA"  => 100
  "per_unit" => true
  "shunt"    => Dict{String, Any}()
  "load"     => Dict{String, Any}("3935"=>Dict{String, Any}("status"=>1, "load_…

Identify non-dispatchable generators:

In [4]:
nuc_series_df = CSV.read("../doc/data/pantagruel_nuc.csv", DataFrame);

In [5]:
nuc_ids = string.(nuc_series_df.id)

44-element Vector{String}:
 "79"
 "87"
 "160"
 "176"
 "179"
 "209"
 "210"
 "232"
 "249"
 "256"
 "257"
 "265"
 "318"
 ⋮
 "723"
 "732"
 "751"
 "757"
 "782"
 "846"
 "908"
 "917"
 "921"
 "943"
 "950"
 "982"

In [6]:
network["nuc"] = Dict{String, Any}(id => network["gen"][id] for id ∈ nuc_ids)

for id ∈ nuc_ids
    delete!(network["gen"], id)
end

In [7]:
Nlines = length(network["branch"])
Nbuses = length(network["bus"])
Nloads = length(network["load"])
Nnucs = length(network["nuc"])
Ngens = length(network["gen"])

Nlines, Nbuses, Nloads, Nnucs, Ngens

(8375, 4097, 3998, 44, 1039)

In [8]:
function sort_ids(data::Dict{String, Any})
    return sort(parse.(Int, keys(data)))
end

sort_ids (generic function with 1 method)

In [9]:
line_ids = sort_ids(network["branch"])
bus_ids = sort_ids(network["bus"])
gen_ids = sort_ids(network["gen"])
nuc_ids = sort_ids(network["nuc"])
load_ids = sort_ids(network["load"]);

In [10]:
(line_ids == collect(1:Nlines), bus_ids == collect(1:Nbuses), gen_ids == collect(1:Ngens), 
    nuc_ids == collect(1:Nnucs), load_ids == collect(1:Nloads))

(true, false, false, false, true)

**Warning**: bus IDs do not go from 1 to N

In [11]:
bus_ids_map = Dict(bus_ids[i] => i for i = 1:Nbuses);
gen_ids_map = Dict(gen_ids[i] => i for i = 1:Ngens)
nuc_ids_map = Dict(nuc_ids[i] => i for i = 1:Nnucs);

In [12]:
gen_ids_map[726]

693

Incidence matrix $M$
(encodes the connection between buses with a direction):

In [13]:
lines_from_bus = [bus_ids_map[network["branch"][string(id)]["f_bus"]] for id ∈ line_ids]
lines_to_bus = [bus_ids_map[network["branch"][string(id)]["t_bus"]] for id ∈ line_ids]

M = sparse([lines_from_bus; lines_to_bus], [1:Nlines; 1:Nlines],
    [-ones(Nlines); ones(Nlines)], Nbuses, Nlines)

4097×8375 SparseMatrixCSC{Float64, Int64} with 16750 stored entries:
⎡⣿⣶⣦⣤⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣾⣾⡷⎤
⎢⠀⠀⠑⠐⠻⠛⠿⠶⢤⣀⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣿⣿⡿⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⢈⠉⣟⡟⣶⣦⣤⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢿⣿⣿⎥
⎢⠀⠀⠀⡀⠀⠀⠀⠀⡒⠒⠋⠀⢀⡁⠉⢙⢛⢲⣤⣀⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣿⣿⣿⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢈⢹⢳⢶⣤⣀⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣿⣿⣯⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠰⠀⠀⠀⠀⠠⠀⠄⠤⠐⠨⠾⠼⠥⣾⠿⠷⢦⣀⡀⠀⠀⠀⠀⠀⠀⠀⣻⣿⣿⎥
⎢⠀⠑⠀⡀⠀⠀⠀⠀⢄⣀⣠⡈⢂⢀⣀⣀⣀⣐⡒⣟⣓⣐⣀⣀⢰⠀⣀⣜⢉⣽⣳⣤⣄⠀⠀⠀⠀⣿⣿⣷⎥
⎢⣀⣠⣤⣤⣀⣐⣠⣀⢈⢈⣩⡀⢈⡀⠀⢈⡡⢡⣀⡍⠍⠉⠉⠉⠁⠉⠈⠉⢉⢈⣤⠀⡹⢷⣶⣄⠀⠷⠷⠞⎥
⎢⠲⡟⢟⢿⢿⣾⣟⣿⣆⣨⣭⣥⢤⣭⣤⣶⣤⡼⣼⠭⢬⣭⡩⠭⢤⡭⣭⣭⣹⠽⢾⡬⣯⣿⣿⣿⣆⠀⠀⠀⎥
⎣⠁⠛⠻⠋⠿⠻⠻⠝⠿⠽⠗⠾⠿⠾⠽⠿⠿⠯⠻⠾⠚⠛⠁⠘⠚⠛⠹⠻⠛⠛⠸⠗⠿⠾⠽⠿⠿⠆⠀⠀⎦

Susceptance matrix:

In [14]:
b = [network["branch"][string(id)]["br_b"] for id ∈ line_ids];

B = M * Diagonal(b)

4097×8375 SparseMatrixCSC{Float64, Int64} with 16750 stored entries:
⎡⣿⣶⣦⣤⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣾⣾⡷⎤
⎢⠀⠀⠑⠐⠻⠛⠿⠶⢤⣀⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣿⣿⡿⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⢈⠉⣟⡟⣶⣦⣤⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢿⣿⣿⎥
⎢⠀⠀⠀⡀⠀⠀⠀⠀⡒⠒⠋⠀⢀⡁⠉⢙⢛⢲⣤⣀⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣿⣿⣿⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢈⢹⢳⢶⣤⣀⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣿⣿⣯⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠰⠀⠀⠀⠀⠠⠀⠄⠤⠐⠨⠾⠼⠥⣾⠿⠷⢦⣀⡀⠀⠀⠀⠀⠀⠀⠀⣻⣿⣿⎥
⎢⠀⠑⠀⡀⠀⠀⠀⠀⢄⣀⣠⡈⢂⢀⣀⣀⣀⣐⡒⣟⣓⣐⣀⣀⢰⠀⣀⣜⢉⣽⣳⣤⣄⠀⠀⠀⠀⣿⣿⣷⎥
⎢⣀⣠⣤⣤⣀⣐⣠⣀⢈⢈⣩⡀⢈⡀⠀⢈⡡⢡⣀⡍⠍⠉⠉⠉⠁⠉⠈⠉⢉⢈⣤⠀⡹⢷⣶⣄⠀⠷⠷⠞⎥
⎢⠲⡟⢟⢿⢿⣾⣟⣿⣆⣨⣭⣥⢤⣭⣤⣶⣤⡼⣼⠭⢬⣭⡩⠭⢤⡭⣭⣭⣹⠽⢾⡬⣯⣿⣿⣿⣆⠀⠀⠀⎥
⎣⠁⠛⠻⠋⠿⠻⠻⠝⠿⠽⠗⠾⠿⠾⠽⠿⠿⠯⠻⠾⠚⠛⠁⠘⠚⠛⠹⠻⠛⠛⠸⠗⠿⠾⠽⠿⠿⠆⠀⠀⎦

By construction, the transpose of the susceptance matrix $B^T$ annihilates the vector $(1, 1, \ldots, 1, 1)$:

In [15]:
B' * ones(Nbuses) == zeros(Int, Nlines)

true

Compute the pseudo-inverse of $B$:

In [16]:
Bi = pinv(Matrix(B))

8375×4097 Matrix{Float64}:
 -0.0225952     5.5164e-6     5.5164e-6    …   5.5164e-6     5.5164e-6
 -0.0225952     5.5164e-6     5.5164e-6        5.5164e-6     5.5164e-6
  1.06328e-5   -0.01088      -0.01088          1.06328e-5    1.06328e-5
  1.06328e-5   -0.01088      -0.01088          1.06328e-5    1.06328e-5
 -2.16281e-5   -2.16281e-5    0.0295151       -2.16281e-5   -2.16281e-5
  1.81255e-6    1.81255e-6   -0.00742419   …   1.81255e-6    1.81255e-6
  7.20936e-6    7.20936e-6    7.20936e-6       7.20936e-6    7.20936e-6
 -3.3705e-5    -3.39816e-5   -3.39816e-5      -1.50774e-5   -1.98764e-5
  1.473e-5      1.48213e-5    1.48213e-5       8.58272e-6    1.01665e-5
 -6.05796e-6   -6.05796e-6   -6.05796e-6      -6.05796e-6   -6.05796e-6
 -6.05796e-6   -6.05796e-6   -6.05796e-6   …  -6.05796e-6   -6.05796e-6
 -8.03995e-7   -1.04703e-6   -1.04703e-6       1.55601e-5    1.13442e-5
 -1.08941e-6   -1.08941e-6   -1.08941e-6      -1.08941e-6   -1.08941e-6
  ⋮                                    

Define $L$ as the inverse of the susceptance matrix multiplied by the susceptance:

In [17]:
L = Diagonal(b) * Bi

8375×4097 Matrix{Float64}:
 -0.499878      0.000122041   0.000122041  …   0.000122041   0.000122041
 -0.499878      0.000122041   0.000122041      0.000122041   0.000122041
  0.000488162  -0.499512     -0.499512         0.000488162   0.000488162
  0.000488162  -0.499512     -0.499512         0.000488162   0.000488162
 -0.000732243  -0.000732243   0.999268        -0.000732243  -0.000732243
  0.000244081   0.000244081  -0.999756     …   0.000244081   0.000244081
  0.000244081   0.000244081   0.000244081      0.000244081   0.000244081
 -0.000752681  -0.000758859  -0.000758859     -0.0003367    -0.000443869
  0.000996762   0.00100294    0.00100294       0.000580781   0.000687951
 -0.000244081  -0.000244081  -0.000244081     -0.000244081  -0.000244081
 -0.000244081  -0.000244081  -0.000244081  …  -0.000244081  -0.000244081
 -2.04378e-5   -2.66158e-5   -2.66158e-5       0.000395543   0.000288374
 -0.000122041  -0.000122041  -0.000122041     -0.000122041  -0.000122041
  ⋮                     

Given the power injections $P_i$ at every bus $i$ and the voltage angles difference $\Delta\theta_a$ on every line $a$, we have

$$P_i = B_{ia} \Delta \theta_a$$

The power flow of each line $a$ is computed by the susceptance times the angle difference:

$$P_a = b_a \Delta\theta_a$$

There is a thermal limit on each line giving the constraint:

$$\left| P_a \right| \leq P_a^\text{therm}$$

For the linearized version of the problem to make sense, we also require the angle differences to be less that 45°:

$$\left| \Delta\theta_a \right| \leq \frac{\pi}{4}$$

This is equivalent to a limit on the line flows, which can in some exceptional cases be more constraining than the thermal limit:

$$\left| P_a \right| \leq\frac{\pi}{4} b_a$$

We define also the maximal power flow to be:

$$P_a^\text{max} = \min\left(P_a^\text{therm}, \frac{\pi}{4} b_a \right)$$

In [18]:
P_max_angle = π/4 * b;

In [19]:
lines_thermal_limit = [network["branch"][string(id)]["rate_a"] for id ∈ line_ids]

count(lines_thermal_limit .> P_max_angle) / Nlines

0.007880597014925373

In [20]:
P_max = min.(lines_thermal_limit, P_max_angle)

8375-element Vector{Float64}:
  4.9
  4.9
  4.9
  4.9
  4.9
  4.9
  4.9
  4.9
  4.9
  4.9
  4.9
  4.9
  4.9
  ⋮
 10.0
 10.0
 10.0
 10.0
  6.902376728174339
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0

## Import generation costs

In [21]:
T = 24 * 364

8736

In [22]:
gen_costs_df = CSV.read("../doc/data/pantagruel_gen_cost_series.csv", DataFrame)
sort!(gen_costs_df)

Row,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,⋯
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,1,6420.33,7164.09,7224.31,6326.79,6860.22,6381.81,6150.76,6527.41,6057.64,6799.47,6332.11,5850.79,6104.69,5744.63,6434.37,5864.33,5559.73,6176.66,5422.5,5484.75,6008.37,6061.61,6415.59,6432.58,6112.82,6950.34,7115.34,6331.59,6985.31,6630.96,6524.66,7023.62,6670.51,7520.31,7149.33,6750.29,7070.27,6758.48,7477.66,6917.79,6604.26,7193.94,6395.57,6398.45,6849.81,6820.46,7084.32,7006.52,6590.17,7332.03,7404.8,6534.45,7108.96,6684.13,6516.98,6965.05,6570.97,7389.17,6995.11,6580.28,6890.36,6573.02,7289.42,6728.01,6412.81,6999.5,6195.93,6190.81,6631.14,6587.83,6835.23,6739.24,6303.94,7027.28,7083.28,6199.26,6764.52,6336.09,6172.01,6630.66,6255.17,7100.19,6741.06,6368.85,6728.5,6466.66,7243.28,6745.4,6495.55,7147.89,6408.78,6465.57,6964.0,6973.99,7269.1,7214.7,6814.62,7566.78,7645.43,⋯
2,2,9832.76,10111.0,9552.12,10115.2,10454.1,10226.5,10767.5,9301.43,9833.6,10696.6,10273.7,9785.72,9106.68,9809.36,9295.27,9652.93,9950.95,9430.7,9989.59,9537.8,10102.7,9600.17,9141.42,9880.92,9520.58,9788.88,9231.31,9805.59,10164.2,9962.68,10534.2,9100.75,9665.39,10558.5,10161.1,9692.22,9024.39,9729.18,9207.38,9547.3,9817.8,9261.07,9775.74,9273.72,9784.48,9226.24,8712.85,9401.48,8996.73,9229.7,8648.22,9212.06,9575.27,9394.58,10003.6,8624.34,9259.1,10236.9,9937.04,9575.94,9023.29,9847.41,9445.58,9902.46,10283.3,9826.77,10428.3,9996.84,10559.6,10033.0,9529.79,10206.7,9768.63,9947.74,9293.56,9768.21,10028.6,9734.93,10224.4,8722.83,9236.56,10098.5,9691.47,9235.32,8602.54,9363.77,8917.95,9350.74,9727.66,9286.95,9922.9,9542.83,10171.5,9722.71,9305.96,10074.7,9730.35,10001.6,9434.79,⋯
3,3,999.425,945.82,903.441,995.912,1030.02,1008.31,1044.74,1015.69,1050.2,1037.38,960.008,1036.46,1037.04,1031.33,1023.68,909.292,949.44,988.297,924.503,938.76,948.023,902.649,895.667,938.156,955.482,887.299,832.17,913.928,939.473,911.38,943.526,912.145,946.056,934.11,858.752,938.049,941.954,939.756,935.524,824.237,866.999,907.892,845.533,860.676,870.393,825.217,818.403,861.287,879.508,812.991,760.555,846.251,877.144,855.916,896.477,875.016,920.229,920.768,858.812,952.097,970.21,982.039,991.242,892.4,946.243,996.525,941.597,962.018,974.754,930.316,922.03,961.389,974.273,900.913,840.495,917.469,939.326,909.169,941.314,912.317,951.186,946.821,881.567,973.247,991.442,1004.95,1017.28,922.754,981.828,1037.92,989.038,1015.38,1033.56,993.803,989.156,1030.92,1044.83,971.08,908.863,⋯
4,4,1061.44,1015.59,1107.72,1173.75,1109.79,1148.47,1226.57,1149.64,1080.17,1130.12,1080.55,979.072,1019.11,1037.44,995.589,1019.18,1061.38,1051.45,1002.62,994.52,1005.61,939.581,873.611,905.007,893.368,833.955,915.157,973.13,904.142,940.899,1020.17,947.368,884.816,944.182,906.184,818.06,872.804,906.76,881.016,920.79,978.874,984.212,949.779,955.027,978.305,923.296,867.213,907.484,903.88,851.88,940.108,1004.98,942.987,987.063,1074.14,1009.76,956.273,1025.33,997.544,919.966,985.333,1029.67,1013.69,1062.22,1127.61,1138.4,1107.16,1112.

In [23]:
gen_costs = Matrix(gen_costs_df[gen_ids, 2:end]) / 2000.

1039×8736 Matrix{Float64}:
 3.21017   3.58205   3.61216   3.1634    …  3.28556   3.43589   3.41063
 4.91638   5.05548   4.77606   5.05758      4.90492   4.69875   5.08561
 0.499713  0.47291   0.45172   0.497956     0.447596  0.453108  0.482992
 0.530718  0.507793  0.55386   0.586875     0.527294  0.503987  0.528598
 2.9711    2.90041   3.04316   3.26441      2.82613   3.02974   3.10334
 3.89398   3.9714    3.51974   3.182     …  3.84782   3.32662   3.13763
 4.19712   4.10996   4.11562   4.24673      4.03415   3.93282   3.93114
 5.7459    5.45858   5.72334   5.49262      6.27343   5.80659   5.46804
 1.16123   1.14622   1.196     1.22513      1.25965   1.15643   1.1263
 1.18171   1.32422   1.36375   1.20134      1.19258   1.20763   1.28572
 6.10334   5.38648   5.66843   5.81139   …  6.33001   5.72106   6.554
 3.49812   3.45964   3.52272   3.63168      3.55152   3.69087   3.70801
 3.05034   3.328     2.83591   3.18433      2.77265   3.28109   2.95878
 ⋮                                    

Distribution of generators on buses:

In [24]:
gen_buses = [bus_ids_map[network["gen"][string(id)]["gen_bus"]] for id ∈ gen_ids]

A_gen = sparse(gen_buses, 1:Ngens, ones(Ngens), Nbuses, Ngens)

4097×1039 SparseMatrixCSC{Float64, Int64} with 1039 stored entries:
⎡⢳⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⠀⡅⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠹⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⢳⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠸⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⢹⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠈⣇⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠸⡄⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠙⣆⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠘⣆⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠘⡄⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⢇⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⢸⡀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⢧⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠸⡄⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢹⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⡇⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢹⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠸⡄⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡇⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡇⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡇⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠱⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠸⎦

Maximum capacity for each generator:

In [25]:
gen_pmax = [convert(Float64, network["gen"][string(id)]["pmax"]) for id ∈ gen_ids]

1039-element Vector{Float64}:
  1.14
  1.6
  0.88
  0.6
  0.27
  0.23
  0.24
  8.45
  3.0
  5.5
  3.0
  3.33600006
  2.1
  ⋮
  0.114
  0.16699999999999998
  0.12
  0.14
 15.8539
  1.05
  0.183
  0.19699999999999998
  0.18600000000000003
  0.256
  0.2475
  0.384

## Import nuclear profiles

In [26]:
nuclear_profiles = Matrix(nuc_series_df[:, 2:end]) / 100.

44×8736 Matrix{Float64}:
 19.7042   19.7052   19.7062   19.7062   …  19.7042   19.7052   19.7052
 10.624    10.623    10.629    10.629       10.633    10.63     10.624
  9.75114   9.75404   9.75791   9.75888     10.4758    9.80338   9.75114
  9.294     9.279     9.291     9.293        9.299     9.296     9.296
 19.1431   19.1431   19.1431   19.1431      19.1431   19.1431   19.1431
 12.7429   12.736    12.477    12.7685   …  12.7764   12.7321   12.7311
 12.0824   12.0824   12.0787   12.0787      12.1026   12.1024   12.0844
 13.64     13.6173   13.553    13.53        13.6965   13.6178   13.635
 12.7271   12.7467   12.7437   12.7301      12.9856   12.9398   12.9096
 12.0348   12.0338   12.0309   12.023       12.0309   12.0328   12.0348
 12.8447   12.8466   12.8437   12.8437   …  12.8234   12.8251   12.8417
 14.0004   13.997    13.8992   13.7054      14.0317   14.0294   14.0088
 17.21     17.21     17.21     17.21        17.26     17.26     17.21
  ⋮                                      ⋱ 

In [27]:
nuc_buses = [bus_ids_map[network["nuc"][string(id)]["gen_bus"]] for id ∈ nuc_ids]

A_nuc = sparse(nuc_buses, 1:Nnucs, ones(Nnucs), Nbuses, Nnucs)

4097×44 SparseMatrixCSC{Float64, Int64} with 44 stored entries:
⎡⠀⠀⎤
⎢⠃⠀⎥
⎢⠀⠀⎥
⎢⠁⠀⎥
⎢⠁⠀⎥
⎢⠆⠀⎥
⎢⠸⠀⎥
⎢⢀⠀⎥
⎢⢀⠀⎥
⎢⠰⠀⎥
⎢⠠⠀⎥
⎢⠈⠆⎥
⎢⠀⡂⎥
⎢⠀⡆⎥
⎢⠀⠆⎥
⎢⠀⠐⎥
⎢⠀⠠⎥
⎢⠀⠰⎥
⎢⠀⠐⎥
⎢⠀⠈⎥
⎢⠀⠀⎥
⎢⠀⠀⎥
⎢⠀⠀⎥
⎢⠀⠀⎥
⎢⠀⠀⎥
⎣⠀⠀⎦

In [28]:
avg_nuclear = sum(nuclear_profiles, dims=2) / T

44×1 Matrix{Float64}:
 18.07435324434912
 10.52281204212462
 16.740458566186987
 12.772566391941368
 12.115955164116976
  9.503013685447213
 12.653122996045495
 11.144890212912095
 11.818796142967246
 10.666854506547292
 10.062367679398305
 12.784686302117901
  9.543592032967025
  ⋮
 35.7308990489626
 36.51960794413923
 26.428929067784257
 18.913049255210407
 16.863302806091106
  4.221288303765866
  6.948160097272817
  3.3505223372592856
  9.220073416218261
  3.427495421245371
 12.033299969955593
 22.566314102564032

In [29]:
total_nuclear = sum(nuclear_profiles, dims=1)[1, :]

8736-element Vector{Float64}:
 881.9648751409237
 881.7314571887553
 881.2714462952173
 881.7447030905244
 881.6503497966312
 884.6994189318873
 893.6933152593614
 894.6164691496289
 895.1912581211678
 895.7077659455551
 896.8124050182465
 896.0096340851055
 896.5861009733926
   ⋮
 888.7747270743923
 888.9280001324361
 889.4097888424648
 888.7765443288088
 887.5665997127107
 886.6996310914843
 884.5294240239839
 884.3841469805394
 882.2464379503751
 882.5014211344914
 881.1820477773415
 881.1772241894436

## Import loads (improper normalization)

In [30]:
loads_df = CSV.read("../doc/data/pantagruel_load_series.csv", DataFrame)
sort!(loads_df)

Row,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,⋯
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,1,74.466,73.2467,73.6876,74.7227,76.3537,79.9334,86.243,90.0782,89.4635,87.3512,84.5539,80.8787,79.5474,81.4733,87.5302,98.8,104.983,103.451,101.351,97.3431,95.5547,90.6984,85.0155,80.2348,78.1799,77.456,78.2365,81.6559,89.8438,99.8647,109.342,113.014,110.815,108.319,104.736,102.629,101.516,102.588,106.338,112.367,116.75,115.626,110.542,106.533,102.838,96.6151,89.0083,85.1835,82.8143,82.2428,82.9679,85.8486,93.7005,103.674,111.34,113.457,111.886,110.302,108.647,107.163,105.635,106.56,108.75,115.145,120.13,119.235,114.852,110.196,106.218,98.7066,90.8857,86.7596,84.0138,83.4976,84.5734,87.0618,95.1262,104.57,113.014,115.734,113.636,110.212,107.312,106.752,106.218,107.359,109.796,115.381,120.178,119.751,115.213,110.762,107.621,100.0,89.1133,83.73,81.3491,79.5543,79.1287,⋯
2,2,54.645,54.7817,54.9209,56.6581,59.2274,64.6843,72.4964,77.7379,78.2035,76.1352,74.4424,72.4959,70.8587,71.2878,75.1507,81.5875,84.6431,82.6045,79.9415,76.0832,72.3888,67.3357,63.1382,60.4706,59.0847,58.3427,59.1942,61.7204,67.9997,76.2182,83.7657,86.6259,83.359,79.8395,76.513,74.4467,72.6973,72.7282,75.0969,80.8904,85.59,84.9328,81.7895,77.7412,73.7495,68.3725,64.1545,61.398,59.2796,58.9786,59.3936,61.718,68.0697,75.3056,82.7403,85.6074,83.6524,82.3887,80.2519,78.6115,77.691,77.9603,79.3298,83.3686,85.3793,83.4385,79.7306,75.3251,72.4849,66.9388,61.8145,58.195,56.6854,55.9693,56.2934,58.6665,63.6301,70.8857,77.2995,79.8957,77.5864,75.8236,74.2967,73.8429,74.1792,75.0823,76.4879,80.51,83.1111,80.4617,76.3632,72.5492,70.6721,66.7525,58.9432,55.1585,53.6654,51.9167,52.1529,⋯
3,3,35.4111,35.0507,35.6161,36.7346,38.4769,41.3568,46.524,50.3135,50.59,49.4542,47.6137,46.4347,45.8693,45.9344,48.5419,53.1658,54.8629,54.6051,52.8572,50.0925,47.7721,44.4225,40.8401,38.111,37.3218,37.1372,37.6341,39.5951,44.8317,50.4485,54.9531,56.6759,54.6138,52.7026,50.4168,49.3233,48.6305,48.9601,50.5719,54.5567,57.4136,57.0236,55.4772,53.034,50.6501,46.8288,43.7725,41.8449,40.9422,40.5072,41.0885,42.5775,47.215,52.424,56.317,57.0835,54.6675,52.8913,50.4698,49.1895,48.3186,48.58,50.6875,54.6642,57.7225,57.2891,55.2437,52.7472,51.0863,47.592,44.1192,41.5723,40.4099,40.0426,40.783,42.2544,46.49,51.1566,54.7616,55.4441,53.4755,51.5137,49.355,48.8399,48.0579,48.1109,49.7224,53.7141,56.7219,56.1972,54.5494,52.0917,50.9583,48.1334,43.232,41.4397,40.4154,39.7576,39.9161,⋯
4,4,38.4188,37.6099,37.2637,37.5414,38.7814,40.541,42.2854,43.7605,44.0413,43.5307,42.6414,41.4128,40.9331,41.7179,43.8195,48.3468,51.3518,50.4975,49.1747,47.5959,47.516,45.4377,42.9135,40.6298,39.3943,38.8876,39.0578,40.2662,43.2577,47.0271,50.8863,52.9611,53.1581,52.5832,51.3432,50.4409,50.2673,50.6917,51.9866,54.2505,55.9356,55.6472,53.1985,51.1998,49.7129,47.4869,42.9286,40.6127,39.0819,38.8532,39.0169,40.2036,43.3335,47.4004,50.4927,52.4912,53.0174,53.0921,53.1291,53.0682,52.9277,53.2522,53.4085,55.2192,56.4347,55.8334,53.7764,51.6408,50.086

In [31]:
loads = Matrix(loads_df[:, 2:end]) / 100.0

3998×8736 Matrix{Float64}:
 0.74466    0.732467   0.736876   …  0.85205    0.800011   0.765229
 0.54645    0.547817   0.549209      0.621106   0.587303   0.556486
 0.354111   0.350507   0.356161      0.406689   0.379902   0.360561
 0.384188   0.376099   0.372637      0.451663   0.420222   0.400347
 1.08463    1.05278    1.03875       1.31638    1.20116    1.09667
 0.742567   0.722593   0.721686   …  0.890943   0.831667   0.764127
 1.50559    1.44043    1.40039       1.89973    1.68634    1.54749
 1.00891    0.9829     0.960034      1.21707    1.11978    1.01126
 1.65217    1.58743    1.55572       2.04231    1.82466    1.68361
 0.356675   0.337645   0.3171        0.441983   0.400283   0.35698
 0.691639   0.679676   0.677324   …  0.849481   0.752974   0.715005
 0.513631   0.507922   0.509357      0.585086   0.559866   0.518014
 1.33578    1.30503    1.28922       1.66914    1.50159    1.35161
 ⋮                                ⋱                        ⋮
 0.457044   0.459938   0.467764   

In [32]:
load_buses = [bus_ids_map[network["load"][string(id)]["load_bus"]] for id ∈ load_ids]

A_load = sparse(load_buses, load_ids, ones(Nloads), Nbuses, Nloads)

4097×3998 SparseMatrixCSC{Float64, Int64} with 3998 stored entries:
⎡⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⡄⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⎦

In [33]:
total_load = sum(loads, dims=1)

1×8736 Matrix{Float64}:
 2364.15  2297.78  2248.39  2256.18  …  2808.25  2735.96  2621.31  2473.69

## Calculate total load by country based on capacity and usage

Determine the expected production by country and type based on capacity and usage:

In [34]:
expected_production_by_country = Dict(gen["country"] => 0.0 for gen ∈ values(network["gen"]))

for gen ∈ values(network["gen"])
    expected_production_by_country[gen["country"]] += gen["pexp"]
end

sort(expected_production_by_country, byvalue=true, rev=true)

OrderedCollections.OrderedDict{String, Float64} with 26 entries:
  "DE" => 279.256
  "PL" => 136.378
  "FR" => 129.988
  "IT" => 127.814
  "ES" => 78.738
  "NL" => 59.6867
  "AT" => 57.8544
  "RO" => 48.9335
  "GR" => 48.2076
  "PT" => 42.5945
  "CZ" => 35.501
  "RS" => 29.6354
  "BG" => 19.8427
  "BE" => 17.3629
  "BA" => 13.4466
  "CH" => 11.6612
  "SI" => 11.1836
  "HU" => 10.6986
  "SK" => 9.39437
  "DK" => 7.71758
  "HR" => 6.7509
  "MK" => 5.00832
  "AL" => 4.75198
  "LU" => 3.56473
  "XX" => 2.6395
  "ME" => 2.33425

Export by country:

In [35]:
out_flow_df = CSV.read("pantagruel_out_flow.csv", DataFrame)

Row,country,out_flow
,String3,Float64
1,DE,5663.42
2,SI,127.709
3,HR,-717.664
4,BE,-725.658
5,ES,-1454.95
6,FR,3016.91
7,RS,167.457
8,DK,-282.057
9,BA,425.84


In [36]:
out_flow_by_country = Dict{String, Float64}()
for (country, flow) in eachrow(out_flow_df)
    out_flow_by_country[country] = flow / 100.0
end
out_flow_by_country["XX"] = 0.0

sort(out_flow_by_country, byvalue=true, rev=true)

OrderedCollections.OrderedDict{String, Float64} with 26 entries:
  "DE" => 56.6342
  "FR" => 30.1691
  "CZ" => 12.5113
  "RO" => 6.64282
  "PT" => 5.73057
  "BA" => 4.2584
  "PL" => 2.34598
  "BG" => 1.89876
  "RS" => 1.67457
  "SI" => 1.27709
  "AL" => 0.0488879
  "LU" => 0.0
  "XX" => 0.0
  "ME" => -0.309394
  "MK" => -1.36804
  "DK" => -2.82057
  "CH" => -5.13167
  "SK" => -5.37451
  "HR" => -7.17664
  "BE" => -7.25658
  "GR" => -8.38198
  "HU" => -9.63781
  "NL" => -9.9127
  "AT" => -9.93982
  "ES" => -14.5495
  "IT" => -41.3325

Nuclear production by country:

In [37]:
nuclear_production_by_country = Dict(country => 0.0 for country ∈ keys(expected_production_by_country))
for id ∈ nuc_ids
    country = network["nuc"][string(id)]["country"]
    nuclear_production_by_country[country] += avg_nuclear[nuc_ids_map[id]]
end
nuclear_production_by_country

Dict{String, Float64} with 26 entries:
  "SI" => 0.0
  "ES" => 45.3376
  "GR" => 0.0
  "LU" => 0.0
  "HU" => 16.8633
  "CH" => 22.9463
  "MK" => 0.0
  "RS" => 0.0
  "XX" => 0.0
  "DE" => 90.667
  "AL" => 0.0
  "FR" => 441.027
  "PL" => 0.0
  "IT" => 0.0
  "AT" => 0.0
  "HR" => 0.0
  "PT" => 0.0
  "ME" => 0.0
  "NL" => 4.22129
  "BE" => 46.9143
  "RO" => 11.829
  "BG" => 17.1781
  "CZ" => 24.8885
  "SK" => 16.7945
  "DK" => 0.0
  "BA" => 0.0

Determine annual load by country:

In [38]:
expected_load_by_country = Dict(country => prod + nuclear_production_by_country[country] - out_flow_by_country[country]
    for (country, prod) ∈ expected_production_by_country)

sort(expected_load_by_country, byvalue=true, rev=true)

OrderedCollections.OrderedDict{String, Float64} with 26 entries:
  "FR" => 540.846
  "DE" => 313.289
  "IT" => 169.147
  "ES" => 138.625
  "PL" => 134.032
  "NL" => 73.8207
  "BE" => 71.5338
  "AT" => 67.7943
  "GR" => 56.5896
  "RO" => 54.1198
  "CZ" => 47.8782
  "CH" => 39.7391
  "HU" => 37.1997
  "PT" => 36.8639
  "BG" => 35.122
  "SK" => 31.5634
  "RS" => 27.9608
  "HR" => 13.9275
  "DK" => 10.5382
  "SI" => 9.90647
  "BA" => 9.18824
  "MK" => 6.37635
  "AL" => 4.70309
  "LU" => 3.56473
  "ME" => 2.64364
  "XX" => 2.6395

In [39]:
annual_load = sum(loads, dims=2)

3998×1 Matrix{Float64}:
  6648.30849591262
  5780.98971189365
  3744.6776833951203
  2933.9041279664293
 12903.279338897046
  7514.775953077493
 13685.749977486332
 13083.337484544412
 18767.726695815192
  3893.6146175195977
  7649.001888283338
  8147.7309374378
 16376.578348475297
     ⋮
  4839.694558630272
  6505.801031515837
  1528.090420111857
  4498.329959300439
  3773.117458098835
  7191.353724004487
  7958.192801021348
   252.15232435394037
   338.4932371837163
  2899.3881922842793
   689.871728814763
   847.851616934551

In [40]:
annual_load_by_country = Dict{String, Float64}()

for id ∈ load_ids
    country = network["bus"][string(network["load"][string(id)]["load_bus"])]["country"]
    if !haskey(annual_load_by_country, country)
        annual_load_by_country[country] = 0.0
    end
    annual_load_by_country[country] += annual_load[id]
end

In [41]:
annual_load_by_country["CH"] / T, expected_load_by_country["CH"]

(71.2539678231059, 39.7391434232667)

In [42]:
annual_load_by_country["FR"] / T, expected_load_by_country["FR"]

(523.880531969998, 540.8458502856153)

In [43]:
annual_load_by_country["DE"] / T, expected_load_by_country["DE"]

(563.7847869744847, 313.28889751171255)

In [44]:
sum(values(annual_load_by_country)) / T, sum(values(expected_load_by_country))

(2761.580522148796, 1939.6111574858896)

In [45]:
load_rescaling_by_country = Dict(country => expected_load_by_country[country] / load * (24 * 364)
    for (country, load) ∈ annual_load_by_country)

Dict{String, Float64} with 22 entries:
  "SI" => 0.665674
  "ES" => 0.496542
  "GR" => 0.982475
  "HU" => 0.756153
  "CH" => 0.557711
  "RS" => 0.644535
  "DE" => 0.555689
  "FR" => 1.03238
  "PL" => 0.697469
  "IT" => 0.515665
  "HR" => 0.697955
  "AT" => 0.970264
  "PT" => 0.649692
  "ME" => 0.742805
  "NL" => 0.590722
  "BE" => 0.732463
  "RO" => 0.818354
  "BG" => 0.821313
  "CZ" => 0.648109
  "SK" => 0.982372
  "DK" => 0.277045
  "BA" => 0.732446

In [46]:
load_rescaling = [load_rescaling_by_country[network["bus"][string(network["load"][string(id)]["load_bus"])]["country"]]
    for id ∈ load_ids];

In [47]:
rescaled_loads = load_rescaling .* loads

3998×8736 Matrix{Float64}:
 0.611599   0.601585   0.605206   …  0.6998     0.657059   0.628492
 0.448806   0.44993    0.451073      0.510123   0.48236    0.45705
 0.290836   0.287876   0.29252       0.334019   0.312018   0.296134
 0.315539   0.308895   0.306052      0.370957   0.345134   0.32881
 1.06562    1.03433    1.02055       1.29331    1.18011    1.07745
 0.729553   0.70993    0.709039   …  0.875329   0.817092   0.750736
 1.4792     1.41519    1.37585       1.86644    1.65678    1.52037
 0.991226   0.965675   0.94321       1.19574    1.10016    0.99354
 1.62321    1.55961    1.52846       2.00652    1.79269    1.65411
 0.350424   0.331727   0.311543      0.434238   0.393268   0.350724
 0.679518   0.667765   0.665454   …  0.834594   0.739779   0.702475
 0.50463    0.499021   0.500431      0.574832   0.550055   0.508936
 1.31237    1.28216    1.26663       1.63989    1.47528    1.32792
 ⋮                                ⋱                        ⋮
 0.253974   0.255583   0.259931    

In [48]:
rescaled_total_load = sum(rescaled_loads, dims=1)[1, :]

8736-element Vector{Float64}:
 1709.9968384682475
 1665.1901450652615
 1623.8957702621124
 1625.9016433243446
 1700.9313035716666
 1867.2707666563945
 2050.023778641126
 2174.9785468541977
 2246.8395174925386
 2287.0446814560555
 2312.78568664523
 2326.6538664878562
 2301.8837454250483
    ⋮
 1955.5675225152186
 1910.254795935055
 1890.2394558527826
 1922.9997678410964
 2028.2051374585556
 2107.247891223793
 2121.513039123077
 2070.9082029012425
 1993.1031291791812
 1950.461980508558
 1884.262211288414
 1787.1959051375786

## Define annual generation constraints by generator

In [49]:
slack_gen = 325
for key ∈ ["country", "type", "pmax"]
    println("$key: ", network["gen"][string(slack_gen)][key])
end
print("name: ", network["bus"][string(network["gen"][string(slack_gen)]["gen_bus"])]["name"])

country: PL
type: fossil_brown_lignite
pmax: 48.87
name: Bełchatów

In [50]:
constrained_gens = Vector{Int}()
gen_constraints = Vector{Float64}()
for id ∈ gen_ids
    if id != slack_gen
        push!(constrained_gens, gen_ids_map[id])
        push!(gen_constraints, network["gen"][string(id)]["pexp"])
    end
end
N_gen_constraints = length(gen_constraints)

1038

In [51]:
A_gen_constraints = sparse(1:N_gen_constraints, constrained_gens, ones(N_gen_constraints), N_gen_constraints, Ngens)

1038×1039 SparseMatrixCSC{Float64, Int64} with 1038 stored entries:
⎡⠑⢤⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⠀⠀⠑⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢢⡀⠀⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠢⎦

## Ramp constraints

In [52]:
ramp_gen_ids = sort([parse(Int, id) for (id, gen) ∈ network["gen"] if gen["aggregated_type"] == "coal"])

167-element Vector{Int64}:
    9
   10
   23
   35
   36
   37
   39
   42
   68
   72
  100
  126
  180
    ⋮
  947
  948
  969
  973
  975
  981
 1004
 1005
 1018
 1019
 1061
 1077

In [53]:
N_ramp_constraints = length(ramp_gen_ids)

167

In [54]:
A_gen_ramp = sparse(1:N_ramp_constraints, [gen_ids_map[id] for id ∈ ramp_gen_ids], ones(N_ramp_constraints), N_ramp_constraints, Ngens)

167×1039 SparseMatrixCSC{Float64, Int64} with 167 stored entries:
⎡⠈⠉⠐⠐⠐⠀⠐⠦⢄⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠉⠙⠲⢤⣀⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠁⠀⠉⠉⠁⠀⠀⠀⠈⠀⠀⠀⠒⠒⠒⠲⠤⠀⠤⢤⢀⡀⢀⢀⎦

In [55]:
gen_ramp = 2. * ones(N_ramp_constraints);

## Cost computation

Quadratic cost for each line:

In [56]:
line_costs = 1. ./ lines_thermal_limit;

Quadratic term:

In [57]:
LA = L * A_gen

quadratic_cost = LA' * (line_costs .* LA)

1039×1039 Matrix{Float64}:
  0.586155     0.201792     0.201592    …  -0.057275    0.0602631   0.0602631
  0.201792     0.69641      0.696211       -0.0580143   0.0592317   0.0592317
  0.201592     0.696211     1.10418        -0.0582136   0.0590324   0.0590324
  0.201592     0.696211     0.900093       -0.0582136   0.0590324   0.0590324
  0.128966     0.127872     0.127673       -0.0553122   0.0561581   0.0561581
  0.131182     0.129961     0.129762    …  -0.0551642   0.0564105   0.0564105
  0.127573     0.12675      0.126551       -0.0553062   0.0563286   0.0563286
  0.15602      0.152999     0.152799       -0.0566643   0.0573773   0.0573773
  0.15602      0.152999     0.152799       -0.0566643   0.0573773   0.0573773
  0.15602      0.152999     0.152799       -0.0566643   0.0573773   0.0573773
  0.156096     0.153071     0.152872    …  -0.0566116   0.0574311   0.0574311
  0.155886     0.15285      0.152651       -0.0564959   0.0575183   0.0575183
  0.155886     0.15285      0.152651 

Linear term:

In [58]:
linear_cost = 2 * LA' * (line_costs .* L * (A_nuc * nuclear_profiles - A_load * rescaled_loads))

1039×8736 Matrix{Float64}:
 -48.0213   -47.5507   -48.1641  -49.8699  …  -54.8324  -50.0914  -47.7925
 -49.2491   -48.82     -49.5055  -51.2865     -56.1116  -51.2632  -48.985
 -49.368    -48.9502   -49.6545  -51.4454     -56.2088  -51.3495  -49.0857
 -49.3781   -48.9588   -49.66    -51.4481     -56.2239  -51.363   -49.099
 -43.2518   -42.805    -43.3173  -44.8364     -49.4579  -45.1499  -43.0021
 -44.0037   -43.5291   -44.0206  -45.5503  …  -50.39    -46.0112  -43.7877
 -42.8362   -42.4013   -42.9213  -44.4326     -48.943   -44.6841  -42.5815
 -52.199    -51.5205   -51.9136  -53.6262     -60.2445  -54.97    -51.9946
 -52.199    -51.5205   -51.9136  -53.6262     -60.2445  -54.97    -51.9946
 -52.199    -51.5205   -51.9136  -53.6262     -60.2445  -54.97    -51.9946
 -51.6227   -50.9625   -51.3694  -53.0765  …  -59.5325  -54.3283  -51.4183
 -50.5681   -49.9368   -50.3632  -52.0472     -58.2458  -53.154   -50.3637
 -50.5681   -49.9368   -50.3632  -52.0472     -58.2458  -53.154   -50.3637


## Export the data

In [59]:
DataDrop.store_matrix("PTDF_data/P_max_gen.h5", gen_pmax)

In [60]:
DataDrop.store_matrix("PTDF_data/P_total.h5", rescaled_total_load - total_nuclear)

In [61]:
DataDrop.store_matrix("PTDF_data/A_gen_total.h5", A_gen_constraints)

In [62]:
DataDrop.store_matrix("PTDF_data/gen_total.h5", gen_constraints)

In [63]:
DataDrop.store_matrix("PTDF_data/A_gen_ramp.h5", A_gen_ramp)

In [64]:
DataDrop.store_matrix("PTDF_data/gen_ramp.h5", gen_ramp)

In [65]:
DataDrop.store_matrix("PTDF_data/linear_line_cost.h5", linear_cost)

In [66]:
DataDrop.store_matrix("PTDF_data/linear_gen_cost.h5", gen_costs)

In [67]:
DataDrop.store_matrix("PTDF_data/quadratic_cost.h5", quadratic_cost)

Data used for analysis of results:

In [68]:
DataDrop.store_matrix("PTDF_data/A_gen.h5", A_gen)

In [69]:
DataDrop.store_matrix("PTDF_data/A_load.h5", A_load)
DataDrop.store_matrix("PTDF_data/P_load.h5", rescaled_loads)

In [70]:
DataDrop.store_matrix("PTDF_data/A_nuc.h5", A_nuc)
DataDrop.store_matrix("PTDF_data/P_nuc.h5", nuclear_profiles)

In [71]:
DataDrop.store_matrix("PTDF_data/PTDF_matrix.h5", L)

In [72]:
DataDrop.store_matrix("PTDF_data/thermal_limits.h5", convert(Vector{Float64}, lines_thermal_limit))

In [73]:
DataDrop.store_matrix("PTDF_data/susceptance.h5", b)